In [ ]:
import sys
sys.path.append('../library')
sys.path.append('../library/rtde')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math
import subprocess
import socket 
import os
from quaternion import quaternion
import keyboard


In [ ]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.188.123'
ROBOT_PORT = 30004
config_filename = '../config/pose_pilot_configuration_with_button.xml'

UDP_IP = "127.0.0.1"
UDP_PORT = 5005
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))


keep_running = True
logging.getLogger().setLevel(logging.INFO)

In [ ]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')
button_names, button_types = conf.get_recipe('button')

In [ ]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)
button = con.send_input_setup(button_names, button_types)

In [ ]:
class deltaTransform:
    def __init__(self, original_base, scale, target_base):
        self.o_base = original_base
        self.scale = scale
        self.t_base = target_base
        
    def doTransform(self, original_point):
        # scale and original point delta
        scaled_point = [s*(o-b) for s, b , o in zip(self.scale, self.o_base, original_point)]
        # transform cartesians
        # TODO: make it general, this only works to swap y and z
        scaled_point[0], scaled_point[1], scaled_point[2] = -scaled_point[0], scaled_point[2], scaled_point[1]
        
        t_point = [t+s for t, s in zip(self.t_base, scaled_point)]
        return t_point